# Procesamiento básico de nube de puntos usando Open3D

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/oscar-ramos/robotica-autonoma-python/blob/main/5-Nubes-Puntos/5-3-Procesamiento-basico-open3d.ipynb)

In [ ]:
!pip install -q open3d

In [ ]:
import numpy as np
import open3d as o3d
import plotly.graph_objs as go

In [ ]:
!wget -q https://raw.githubusercontent.com/oscar-ramos/robotica-autonoma-python/main/5-Nubes-Puntos/datos/tabletop.pcd

In [ ]:
# Cargar el archivo .pcd file
cloud = o3d.io.read_point_cloud("tabletop.pcd")

if cloud.has_colors():
    print("La nube de puntos tiene información de colores")

In [ ]:
# Extracción de coordenadas
colors = np.asarray(cloud.colors)
points = np.asarray(cloud.points)
x3d = points[:, 0]
y3d = points[:, 1]
z3d = points[:, 2]

In [ ]:
# Creación de un gráfico de puntos 3D (scatter plot) de Plotly
fig = go.Figure(data=[go.Scatter3d(x=x3d, y=y3d, z=z3d, mode='markers',
                                   marker=dict(size=2, color=colors, opacity=0.6)
                                   )])

# Mitad del rango de los datos
rango = 0.5*np.max([np.ptp(x3d), np.ptp(y3d), np.ptp(z3d)])

# Igualar escalas en cada eje
fig.update_layout(scene=dict(
    xaxis=dict(range=[np.mean(x3d)-rango, np.mean(x3d)+rango], title='X'),
    yaxis=dict(range=[np.mean(y3d)-rango, np.mean(y3d)+rango], title='Y'),
    zaxis=dict(range=[np.mean(z3d)-rango, np.mean(z3d)+rango], title='Z'),
    aspectratio=dict(x=1, y=1, z=1),
    aspectmode='manual'
))

# Mostrar la figura
fig.show()

In [ ]:
print("Número de puntos:", points.shape[0])

In [ ]:
points.shape

## Submuestreo

In [ ]:
# Objeto submuestreado
cloud_filtrado = cloud.voxel_down_sample(voxel_size=0.02)

# Puntos y colores
colors = np.asarray(cloud_filtrado.colors)
points = np.asarray(cloud_filtrado.points)

print("Número de puntos:", points.shape[0])

In [ ]:
x3d = points[:, 0]
y3d = points[:, 1]
z3d = points[:, 2]

In [ ]:
# Creación de un gráfico de puntos 3D (scatter plot) de Plotly
fig = go.Figure(data=[go.Scatter3d(x=x3d, y=y3d, z=z3d, mode='markers',
                                   marker=dict(size=2, color=colors, opacity=0.6)
                                   )])

# Mitad del rango de los datos
rango = 0.5*np.max([np.ptp(x3d), np.ptp(y3d), np.ptp(z3d)])

# Igualar escalas en cada eje
fig.update_layout(scene=dict(
    xaxis=dict(range=[np.mean(x3d)-rango, np.mean(x3d)+rango], title='X'),
    yaxis=dict(range=[np.mean(y3d)-rango, np.mean(y3d)+rango], title='Y'),
    zaxis=dict(range=[np.mean(z3d)-rango, np.mean(z3d)+rango], title='Z'),
    aspectratio=dict(x=1, y=1, z=1),
    aspectmode='manual'
))

# Mostrar la figura
fig.show()

### Filtro de región de interés

In [ ]:
# Crear un bounding box
bbox = o3d.geometry.AxisAlignedBoundingBox(min_bound=(float('-inf'), float('-inf'), 0.5),
                                           max_bound=(float('inf'), float('inf'), 2))

# Crop the point cloud using the bounding box
cloud_roi = cloud.crop(bbox)

In [ ]:
# Puntos y colores
colors = np.asarray(cloud_roi.colors)
points = np.asarray(cloud_roi.points)

x3d = points[:, 0]; y3d = points[:, 1]; z3d = points[:, 2]
# Creación de un gráfico de puntos 3D (scatter plot) de Plotly
fig = go.Figure(data=[go.Scatter3d(x=x3d, y=y3d, z=z3d, mode='markers',
                                   marker=dict(size=2, color=colors, opacity=0.6)
                                   )])
# Mitad del rango de los datos
rango = 0.5*np.max([np.ptp(x3d), np.ptp(y3d), np.ptp(z3d)])
# Igualar escalas en cada eje
fig.update_layout(scene=dict(
    xaxis=dict(range=[np.mean(x3d)-rango, np.mean(x3d)+rango], title='X'),
    yaxis=dict(range=[np.mean(y3d)-rango, np.mean(y3d)+rango], title='Y'),
    zaxis=dict(range=[np.mean(z3d)-rango, np.mean(z3d)+rango], title='Z'),
    aspectratio=dict(x=1, y=1, z=1),
    aspectmode='manual'
))
# Mostrar la figura
fig.show()

### RANSAC

In [ ]:
# Segmentación del plano
plane_model, inliers = cloud_roi.segment_plane(distance_threshold=1, ransac_n=3, num_iterations=1000)

In [ ]:
# Extracción de inliers y outliers
inlier_cloud = cloud_roi.select_by_index(inliers)
outlier_cloud = cloud_roi.select_by_index(inliers, invert=True)